In [1]:
# import libraries
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
import mlflow
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pickle
import mlflow.pyfunc
from sklearn.metrics import mean_squared_error
import numpy as np

# set the tracking uri
MLFLOW_TRACKING_URI = "http://localhost:5000"

# create a client
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
# search experiments
client.search_experiments()

[<Experiment: artifact_location='file:///workspaces/mlops-course/module_2/mlruns/216665436346368280', creation_time=1716473206979, experiment_id='216665436346368280', last_update_time=1716473206979, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716469978584, experiment_id='0', last_update_time=1716469978584, lifecycle_stage='active', name='Default', tags={}>]

In [3]:
# create an experiment
#client.create_experiment(name="test_experiment")

In [4]:
# search runs
runs = client.search_runs(
    experiment_ids="216665436346368280",
    filter_string="metrics.rmse < 5.12",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
# shows the runs
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 011c0a04bf1045d791de5d3eac046d56, rmse: 5.1149
run id: 972a717b23ac4e788bcdda758b0754cd, rmse: 5.1149
run id: 2afd063875b9466ab701042327eeaec2, rmse: 5.1149
run id: c5c30653bc7e4fc49c69c3538c87683b, rmse: 5.1149
run id: 9586f52f48f74f99a90f5b985f1fc0fd, rmse: 5.1149


In [6]:
# set the tracking uri
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
# register the model
run_id = "011c0a04bf1045d791de5d3eac046d56"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-model")

Registered model 'nyc-taxi-model' already exists. Creating a new version of this model...
2024/05/25 01:29:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-model, version 19
Created version '19' of model 'nyc-taxi-model'.


<ModelVersion: aliases=[], creation_timestamp=1716600581479, current_stage='None', description='', last_updated_timestamp=1716600581479, name='nyc-taxi-model', run_id='011c0a04bf1045d791de5d3eac046d56', run_link='', source='file:///workspaces/mlops-course/module_2/mlruns/216665436346368280/011c0a04bf1045d791de5d3eac046d56/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='19'>

In [8]:
model_uri

'runs:/011c0a04bf1045d791de5d3eac046d56/model'

In [9]:
client.search_model_versions("name = 'nyc-taxi-model'")

[<ModelVersion: aliases=[], creation_timestamp=1716600581479, current_stage='None', description='', last_updated_timestamp=1716600581479, name='nyc-taxi-model', run_id='011c0a04bf1045d791de5d3eac046d56', run_link='', source='file:///workspaces/mlops-course/module_2/mlruns/216665436346368280/011c0a04bf1045d791de5d3eac046d56/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='19'>,
 <ModelVersion: aliases=['pre-production'], creation_timestamp=1716586737342, current_stage='Staging', description='This model was transitioned to pre production on 2024-05-25', last_updated_timestamp=1716598379175, name='nyc-taxi-model', run_id='c5c30653bc7e4fc49c69c3538c87683b', run_link='', source='file:///workspaces/mlops-course/module_2/mlruns/216665436346368280/c5c30653bc7e4fc49c69c3538c87683b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>,
 <ModelVersion: aliases=[], creation_timestamp=1716598378889, current_stage='None', description

In [10]:
# get latest version
# sorted(client.search_model_versions(f"name='nyc-taxi-model'"), key=lambda mv: int(mv.version), reverse=True)[0]

# latest_versions = client.get_latest_versions(name = 'nyc-taxi-model')

latest_versions = client.search_model_versions("name='nyc-taxi-model'")

for latest in latest_versions:
    print(f"run id: {latest.run_id}, version: {latest.version}, aliases: {latest.aliases}, tags: {latest.tags}")

run id: 011c0a04bf1045d791de5d3eac046d56, version: 19, aliases: [], tags: {}
run id: c5c30653bc7e4fc49c69c3538c87683b, version: 4, aliases: ['pre-production'], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 18, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 17, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 16, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 15, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 14, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 13, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 12, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 11, aliases: ['testing'], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 10, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 9, aliases: [], tags: {}
run id: 011c0a04bf1045d791de5d3eac046d56, version: 8,

In [11]:
# set the registered model alias
client.set_registered_model_alias(name="nyc-taxi-model", version="4", alias="pre-production")

In [12]:
from datetime import datetime

# update the model version description
client.update_model_version(name='nyc-taxi-model', version='4', description=f'This model was transitioned to pre production on {datetime.today().date()}')

<ModelVersion: aliases=['pre-production'], creation_timestamp=1716586737342, current_stage='Staging', description='This model was transitioned to pre production on 2024-05-25', last_updated_timestamp=1716600581746, name='nyc-taxi-model', run_id='c5c30653bc7e4fc49c69c3538c87683b', run_link='', source='file:///workspaces/mlops-course/module_2/mlruns/216665436346368280/c5c30653bc7e4fc49c69c3538c87683b/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>

In [13]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(run_id, artifact_path, X_test, y_test):
    logged_model = f'runs:/{run_id}/{artifact_path}'
    loaded_model = mlflow.pyfunc.load_model(logged_model)
    y_pred = loaded_model.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    return {"rmse": rmse}

In [14]:
df = read_dataframe("./data/yellow_tripdata_2023-02.parquet")

In [15]:
client.download_artifacts(run_id, path='preprocessor', dst_path='.')

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/workspaces/mlops-course/module_2/preprocessor'

In [16]:
with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [17]:
X_test = preprocess(df, dv)
y_test = df['duration'].values

In [18]:
%time print(test_model(run_id="c5c30653bc7e4fc49c69c3538c87683b", artifact_path="models_artifact", X_test=X_test, y_test=y_test))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [01:29:58] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


{'rmse': 5.4065983014906696}
CPU times: user 6min 41s, sys: 305 ms, total: 6min 41s
Wall time: 3min 40s
